In [1]:
!pip install contextualized-topic-models==2.2.0
!pip install pyldavis

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
!wget https://raw.githubusercontent.com/vinid/data/master/dbpedia_sample_abstract_20k_unprep.txt

--2023-04-30 20:36:05--  https://raw.githubusercontent.com/vinid/data/master/dbpedia_sample_abstract_20k_unprep.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6208417 (5.9M) [text/plain]
Saving to: ‘dbpedia_sample_abstract_20k_unprep.txt.1’

dbpedia_sample_abst 100%[===================>]   5.92M  32.2MB/s    in 0.2s    

2023-04-30 20:36:06 (32.2 MB/s) - ‘dbpedia_sample_abstract_20k_unprep.txt.1’ saved [6208417/6208417]



In [3]:
!head -n 3 dbpedia_sample_abstract_20k_unprep.txt

The Mid-Peninsula Highway is a proposed freeway across the Niagara Peninsula in the Canadian province of Ontario. Although plans for a highway connecting Hamilton to Fort Erie south of the Niagara Escarpment have surfaced for decades,it was not until The Niagara Frontier International Gateway Study was published by the Ministry
Monte Zucker (died March 15, 2007) was an American photographer. He specialized in wedding photography, entering it as a profession in 1947. In the 1970s he operated a studio in Silver Spring, Maryland. Later he lived in Florida. He was Brides Magazine's Wedding Photographer of the Year for 1990 and
Henry Howard, 13th Earl of Suffolk, 6th Earl of Berkshire (8 August 1779 – 10 August 1779) was a British peer, the son of Henry Howard, 12th Earl of Suffolk. His father died on 7 March 1779, leaving behind his pregnant widow. The Earldom of Suffolk became dormant until she


In [4]:
text_file = "dbpedia_sample_abstract_20k_unprep.txt"

In [5]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
import nltk

In [6]:
nltk.download('stopwords')

documents = [line.strip() for line in open(text_file, encoding="utf-8").readlines()]
sp = WhiteSpacePreprocessing(documents, stopwords_language='english')
preprocessed_documents, unpreprocessed_corpus, vocab = sp.preprocess()

[nltk_data] Downloading package stopwords to /home/joon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# 전처리 전 문서
unpreprocessed_corpus[:2]

['The Mid-Peninsula Highway is a proposed freeway across the Niagara Peninsula in the Canadian province of Ontario. Although plans for a highway connecting Hamilton to Fort Erie south of the Niagara Escarpment have surfaced for decades,it was not until The Niagara Frontier International Gateway Study was published by the Ministry',
 "Monte Zucker (died March 15, 2007) was an American photographer. He specialized in wedding photography, entering it as a profession in 1947. In the 1970s he operated a studio in Silver Spring, Maryland. Later he lived in Florida. He was Brides Magazine's Wedding Photographer of the Year for 1990 and"]

In [8]:
# normalization 전처리 후 문서
preprocessed_documents[:2]

['mid peninsula highway proposed across peninsula canadian province ontario although highway connecting hamilton fort south international study published ministry',
 'died march american photographer specialized photography operated studio silver spring maryland later lived florida magazine photographer year']

In [9]:
# 전체 단어 집합의 크기
print('bag of words에 사용 될 단어 집합의 크기 :',len(vocab))

bag of words에 사용 될 단어 집합의 크기 : 2000


In [ ]:
tp = TopicModelDataPreparation("paraphrase-distilroberta-base-v1")
training_dataset = tp.fit(text_for_contextual=unpreprocessed_corpus, text_for_bow=preprocessed_documents)

Batches:   0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
len(tp.vocab)

In [ ]:
# CTM학습하기 
ctm = CombinedTM(bow_size=len(tp.vocab), contextual_size=768, n_components=50, num_epochs=20)
ctm.fit(training_dataset)

In [ ]:
# 결과 출력
ctm.get_topic_lists(5)

In [ ]:
# 시각화
lda_vis_data = ctm.get_ldavis_data_format(tp.vocab, training_dataset, n_samples=10)

In [ ]:
import pyLDAvis as vis

lda_vis_data = ctm.get_ldavis_data_format(tp.vocab, training_dataset, n_samples=10)

ctm_pd = vis.prepare(**lda_vis_data)
vis.display(ctm_pd)

In [ ]:
# 예측
import numpy as np

topics_predictions = ctm.get_thetas(training_dataset, n_samples=5) # get all the topic predictions

In [ ]:
# 전처리 문서의 첫번째 문서
print(preprocessed_documents[0])

In [ ]:
# 모델 저장 및 로드
ctm.save(models_dir="./")

In [ ]:
del ctm

In [ ]:
ctm = CombinedTM(bow_size=len(tp.vocab), contextual_size=768, num_epochs=100, n_components=50)

ctm.load("/content/contextualized_topic_model_nc_50_tpm_0.0_tpv_0.98_hs_prodLDA_ac_(100, 100)_do_softplus_lr_0.2_mo_0.002_rp_0.99",
                                                                                                      epoch=19)

In [ ]:
ctm.get_topic_lists(5)